In [53]:
import pandas as pd
import numpy as np

In [20]:
df=pd.read_csv('traffic/traffic_victoria_road.csv')

In [165]:
import requests
import json

# API URL for querying the first 5 results from the dataset
url = 'https://opendata.transport.nsw.gov.au/api/3/action/datastore_search?resource_id=b8bd742f-af37-4999-887f-6d3657349d13&limit=5'

# Make the request
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()
    
    # Pretty print the JSON response
    print(json.dumps(data, indent=4))
    
    # Access specific data within the response (e.g., the records)
    if 'result' in data and 'records' in data['result']:
        records = data['result']['records']
        # Print each record
        for record in records:
            print(record)
else:
    print(f"Error: {response.status_code}")
    print(response.text)


{
    "help": "https://opendata.transport.nsw.gov.au/api/3/action/help_show?name=datastore_search",
    "success": true,
    "result": {
        "include_total": true,
        "limit": 5,
        "records_format": "objects",
        "resource_id": "b8bd742f-af37-4999-887f-6d3657349d13",
        "total_estimation_threshold": null,
        "records": [
            {
                "_id": 1,
                "<html>": "<head><title>504 Gateway Time-out</title></head>"
            },
            {
                "_id": 2,
                "<html>": "<body>"
            },
            {
                "_id": 3,
                "<html>": "<center><h1>504 Gateway Time-out</h1></center>"
            },
            {
                "_id": 4,
                "<html>": "</body>"
            },
            {
                "_id": 5,
                "<html>": "</html>"
            }
        ],
        "fields": [
            {
                "id": "_id",
                "type": "int"
          

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9513 entries, 0 to 9512
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   year                    9513 non-null   int64  
 1   date                    9513 non-null   object 
 2   cardinal_direction_seq  9513 non-null   object 
 3   classification_seq      9513 non-null   object 
 4   hour_00                 9339 non-null   float64
 5   hour_01                 9302 non-null   float64
 6   hour_02                 9281 non-null   float64
 7   hour_03                 9318 non-null   float64
 8   hour_04                 9388 non-null   float64
 9   hour_05                 9475 non-null   float64
 10  hour_06                 9475 non-null   float64
 11  hour_07                 9476 non-null   float64
 12  hour_08                 9473 non-null   float64
 13  hour_09                 9471 non-null   float64
 14  hour_10                 9467 non-null   

In [22]:
df.head()

,year,date,cardinal_direction_seq,classification_seq,hour_00,hour_01,hour_02,hour_03,hour_04,hour_05,...,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,public_holiday,school_holiday
0,2015,2015-03-31,Eastbound,Heavy Vehicles,NaN,NaN,NaN,NaN,NaN,NaN,...,140.0,102.0,86.0,53.0,46.0,43.0,24.0,19.0,0,0
1,2015,2015-03-31,Eastbound,Light Vehicles,NaN,NaN,NaN,NaN,NaN,NaN,...,1834.0,2028.0,1955.0,1431.0,1099.0,993.0,755.0,422.0,0,0
2,2015,2015-03-31,Eastbound,All Vehicles,NaN,NaN,NaN,NaN,NaN,NaN,...,1974.0,2130.0,2041.0,1484.0,1145.0,1036.0,779.0,441.0,0,0
3,2015,2015-04-01,Eastbound,Heavy Vehicles,18.0,11.0,8.0,4.0,29.0,81.0,...,143.0,100.0,91.0,57.0,49.0,35.0,28.0,25.0,0,0
4,2015,2015-04-01,Eastbound,Light Vehicles,211.0,113.0,92.0,113.0,239.0,973.0,...,2008.0,2123.0,2198.0,1657.0,1144.0,1052.0,883.0,533.0,0,0


In [25]:
# Step 1: Melt the DataFrame to convert the hour columns into rows
df_melted = pd.melt(df, 
                    id_vars=['year', 'date', 'cardinal_direction_seq', 'classification_seq', 'public_holiday', 'school_holiday'],
                    value_vars=[f'hour_{str(i).zfill(2)}' for i in range(24)],
                    var_name='hour', 
                    value_name='traffic_count')

# Step 2: Clean up the 'hour' column (convert 'hour_00' to '00', 'hour_01' to '01', etc.)
df_melted['hour'] = df_melted['hour'].str.replace('hour_', '').astype(int)

# Step 3: Pivot the DataFrame to separate 'Heavy Vehicles' and 'Light Vehicles' into their own columns
df_pivoted = df_melted.pivot_table(index=['year', 'date', 'hour', 'public_holiday', 'school_holiday'],
                                   columns='classification_seq', 
                                   values='traffic_count').reset_index()

# Step 4: Rename the columns for clarity
df_pivoted.columns.name = None  # Remove the pivot table's automatic column grouping name
df_pivoted.rename(columns={'Heavy Vehicles': 'heavy_vehicle', 'Light Vehicles': 'light_vehicle'}, inplace=True)

# Step 5: Display the transformed DataFrame


   year        date  hour  public_holiday  school_holiday  All Vehicles  \
0  2015  2015-03-31    10               0               0        2118.0   
1  2015  2015-03-31    11               0               0        1774.0   
2  2015  2015-03-31    12               0               0        1794.0   
3  2015  2015-03-31    13               0               0        1628.0   
4  2015  2015-03-31    14               0               0        1592.0   

   heavy_vehicle  light_vehicle  
0          134.0         1984.0  
1          174.0         1600.0  
2          116.0         1678.0  
3          122.0         1506.0  
4          130.0         1462.0  


In [29]:
df_pivoted.tail(100)

,year,date,hour,public_holiday,school_holiday,All Vehicles,heavy_vehicle,light_vehicle
75149,2024,2024-08-27,20,0,0,447.0,38.0,409.0
75150,2024,2024-08-27,21,0,0,345.0,31.0,314.0
75151,2024,2024-08-27,22,0,0,288.0,29.0,259.0
75152,2024,2024-08-27,23,0,0,122.0,15.0,107.0
75153,2024,2024-08-28,0,0,0,78.0,11.0,67.0
...,...,...,...,...,...,...,...,...
75244,2024,2024-08-31,19,0,0,715.0,46.0,669.0
75245,2024,2024-08-31,20,0,0,572.0,31.0,541.0
75246,2024,2024-08-31,21,0,0,895.0,32.0,863.0
75247,2024,2024-08-31,22,0,0,1053.0,45.0,1008.0


In [28]:
df_pivoted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75249 entries, 0 to 75248
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   year            75249 non-null  int64  
 1   date            75249 non-null  object 
 2   hour            75249 non-null  int64  
 3   public_holiday  75249 non-null  int64  
 4   school_holiday  75249 non-null  int64  
 5   All Vehicles    75249 non-null  float64
 6   heavy_vehicle   75155 non-null  float64
 7   light_vehicle   75238 non-null  float64
dtypes: float64(3), int64(4), object(1)
memory usage: 4.6+ MB


In [77]:
df_clean=df_pivoted.dropna()
daily_traffic_data = df_clean.resample('D').agg({
    'public_holiday': 'max',  # Taking the maximum value for the day
# Step 1: Create a list of school term date ranges for 2020 to 2025
school_term_dates = [
    # 2020 Term dates (Eastern and Western division)
    ('2020-01-28', '2020-04-09'),  # Term 1 Eastern
    ('2020-02-04', '2020-04-09'),  # Term 1 Western
    ('2020-04-27', '2020-07-03'),  # Term 2
    ('2020-07-20', '2020-09-25'),  # Term 3
    ('2020-10-12', '2020-12-18'),  # Term 4
    # 2021 Term dates
    ('2021-01-27', '2021-04-01'),  # Term 1 Eastern
    ('2021-02-03', '2021-04-01'),  # Term 1 Western
    ('2021-04-19', '2021-06-25'),  # Term 2
    ('2021-07-12', '2021-09-17'),  # Term 3
    ('2021-10-05', '2021-12-17'),  # Term 4
    # 2022 Term dates
    ('2022-01-28', '2022-04-08'),  # Term 1 Eastern
    ('2022-02-04', '2022-04-08'),  # Term 1 Western
    ('2022-04-26', '2022-07-01'),  # Term 2
    ('2022-07-18', '2022-09-23'),  # Term 3
    ('2022-10-10', '2022-12-20'),  # Term 4
    # 2023 Term dates
    ('2023-01-27', '2023-04-06'),  # Term 1 Eastern
    ('2023-02-03', '2023-04-06'),  # Term 1 Western
    ('2023-04-24', '2023-06-30'),  # Term 2
    ('2023-07-17', '2023-09-22'),  # Term 3
    ('2023-10-09', '2023-12-19'),  # Term 4
    # 2024 Term dates
    ('2024-02-01', '2024-04-12'),  # Term 1 Eastern
    ('2024-02-08', '2024-04-12'),  # Term 1 Western
    ('2024-04-30', '2024-07-05'),  # Term 2
    ('2024-07-23', '2024-09-27'),  # Term 3
    ('2024-10-14', '2024-12-18'),  # Term 4
    # 2025 Term dates
    ('2025-02-04', '2025-04-11'),  # Term 1 Eastern
    ('2025-02-11', '2025-04-11'),  # Term 1 Western
    ('2025-04-30', '2025-07-04'),  # Term 2
    ('2025-07-22', '2025-09-26'),  # Term 3
    ('2025-10-13', '2025-12-19')   # Term 4
]

# Step 2: Create a DataFrame for school term dates with 0 as school days
df_school_term = pd.DataFrame({
    'Start': pd.to_datetime([start for start, end in school_term_dates]),
    'End': pd.to_datetime([end for start, end in school_term_dates]),
    'school_holiday': 0  # School days are 0
})

# Step 3: Create a date range DataFrame from 2020 to 2025
date_range = pd.date_range(start='2020-01-01', end='2025-12-31', freq='D')
df_dates = pd.DataFrame(date_range, columns=['datetime'])
df_dates['school_holiday'] = 1  # Default to 1 (holidays)

# Step 4: Mark the term dates (non-holidays) as 0
for _, row in df_school_term.iterrows():
    df_dates.loc[(df_dates['datetime'] >= row['Start']) & (df_dates['datetime'] <= row['End']), 'school_holiday'] = 0

# Step 5: Data for public holidays from 2020 to 2025
public_holidays = {
    'holiday': [
        "New Year's Day", "Australia Day", "Good Friday", "Easter Saturday", "Easter Sunday", 
        "Easter Monday", "Anzac Day", "King's Birthday", "Labour Day", "Christmas Day", "Boxing Day"
    ],
    '2020': [
        '2020-01-01', '2020-01-27', '2020-04-10', '2020-04-11', '2020-04-12', 
        '2020-04-13', '2020-04-25', '2020-06-08', '2020-10-05', '2020-12-25', '2020-12-26'
    ],
    '2021': [
        '2021-01-01', '2021-01-26', '2021-04-02', '2021-04-03', '2021-04-04', 
        '2021-04-05', '2021-04-25', '2021-06-14', '2021-10-04', '2021-12-25', '2021-12-26'
    ],
    '2022': [
        '2022-01-01', '2022-01-26', '2022-04-15', '2022-04-16', '2022-04-17', 
        '2022-04-18', '2022-04-25', '2022-06-13', '2022-10-03', '2022-12-25', '2022-12-26'
    ],
    '2023': [
        '2023-01-01', '2023-01-26', '2023-04-07', '2023-04-08', '2023-04-09', 
        '2023-04-10', '2023-04-25', '2023-06-12', '2023-10-02', '2023-12-25', '2023-12-26'
    ],
    '2024': [
        '2024-01-01', '2024-01-26', '2024-03-29', '2024-03-30', '2024-03-31', 
        '2024-04-01', '2024-04-25', '2024-06-10', '2024-10-07', '2024-12-25', '2024-12-26'
    ],
    '2025': [
        '2025-01-01', '2025-01-27', '2025-04-18', '2025-04-19', '2025-04-20', 
        '2025-04-21', '2025-04-25', '2025-06-09', '2025-10-06', '2025-12-25', '2025-12-26'
    ]
}

# Step 6: Convert public holiday data to a DataFrame
df_holidays = pd.DataFrame(public_holidays)
df_holidays_melted = df_holidays.melt(id_vars=['holiday'], var_name='Year', value_name='Date')
df_holidays_melted = df_holidays_melted.dropna()
df_holidays_melted['Date'] = pd.to_datetime(df_holidays_melted['Date'])
df_holidays_melted['public_holiday'] = 1

# Step 7: Merge public holidays into the date range DataFrame
df_final = pd.merge(df_dates, df_holidays_melted[['Date', 'public_holiday']], left_on='datetime', right_on='Date', how='left')
df_final['public_holiday'] = df_final['public_holiday'].fillna(0).astype(int)

# Step 8: Drop unnecessary 'Date' column
df_holiday = df_final.drop(columns=['Date'])

# Step 9: Find the first date in df_holiday
first_date = df_holiday.index.min()

# Step 10: Filter df_final to include only dates after the first date in df_holiday
df_holiday_append = df_holiday.set_index('datetime')

# Step 11: Append the filtered holiday data to daily_traffic_data
daily_traffic_data.update(df_holiday_append[['school_holiday', 'public_holiday']])



# Drop 'Date' column
daily_traffic_data=daily_traffic_data.drop(columns=['Date'])  # Display the last few rows of the updated data

# Step 3: Overwrite 'school_holiday' and 'public_holiday' in daily_traffic_data using df_holiday
#daily_traffic_data.update(df_holiday[['school_holiday', 'public_holiday']])

   'school_holiday': 'max',  # Taking the maximum value for the day
    'heavy_vehicle': 'sum',   # Summing heavy vehicle counts
    'light_vehicle': 'sum'    # Summing light vehicle counts
})

In [155]:
import pandas as pd

# Step 1: Create a list of school term date ranges for 2020 to 2025
school_term_dates = [
    # 2020 Term dates (Eastern and Western division)
    ('2020-01-28', '2020-04-09'),  # Term 1 Eastern
    ('2020-02-04', '2020-04-09'),  # Term 1 Western
    ('2020-04-27', '2020-07-03'),  # Term 2
    ('2020-07-20', '2020-09-25'),  # Term 3
    ('2020-10-12', '2020-12-18'),  # Term 4
    # 2021 Term dates
    ('2021-01-27', '2021-04-01'),  # Term 1 Eastern
    ('2021-02-03', '2021-04-01'),  # Term 1 Western
    ('2021-04-19', '2021-06-25'),  # Term 2
    ('2021-07-12', '2021-09-17'),  # Term 3
    ('2021-10-05', '2021-12-17'),  # Term 4
    # 2022 Term dates
    ('2022-01-28', '2022-04-08'),  # Term 1 Eastern
    ('2022-02-04', '2022-04-08'),  # Term 1 Western
    ('2022-04-26', '2022-07-01'),  # Term 2
    ('2022-07-18', '2022-09-23'),  # Term 3
    ('2022-10-10', '2022-12-20'),  # Term 4
    # 2023 Term dates
    ('2023-01-27', '2023-04-06'),  # Term 1 Eastern
    ('2023-02-03', '2023-04-06'),  # Term 1 Western
    ('2023-04-24', '2023-06-30'),  # Term 2
    ('2023-07-17', '2023-09-22'),  # Term 3
    ('2023-10-09', '2023-12-19'),  # Term 4
    # 2024 Term dates
    ('2024-02-01', '2024-04-12'),  # Term 1 Eastern
    ('2024-02-08', '2024-04-12'),  # Term 1 Western
    ('2024-04-30', '2024-07-05'),  # Term 2
    ('2024-07-23', '2024-09-27'),  # Term 3
    ('2024-10-14', '2024-12-18'),  # Term 4
    # 2025 Term dates
    ('2025-02-04', '2025-04-11'),  # Term 1 Eastern
    ('2025-02-11', '2025-04-11'),  # Term 1 Western
    ('2025-04-30', '2025-07-04'),  # Term 2
    ('2025-07-22', '2025-09-26'),  # Term 3
    ('2025-10-13', '2025-12-19')   # Term 4
]

# Step 2: Create a DataFrame for school term dates with 0 as school days
df_school_term = pd.DataFrame({
    'Start': pd.to_datetime([start for start, end in school_term_dates]),
    'End': pd.to_datetime([end for start, end in school_term_dates]),
    'school_holiday': 0  # School days are 0
})

# Step 3: Create a date range DataFrame from 2020 to 2025
date_range = pd.date_range(start='2020-01-01', end='2025-12-31', freq='D')
df_dates = pd.DataFrame(date_range, columns=['datetime'])
df_dates['school_holiday'] = 1  # Default to 1 (holidays)

# Step 4: Mark the term dates (non-holidays) as 0
for _, row in df_school_term.iterrows():
    df_dates.loc[(df_dates['datetime'] >= row['Start']) & (df_dates['datetime'] <= row['End']), 'school_holiday'] = 0

# Step 5: Data for public holidays from 2020 to 2025
public_holidays = {
    'holiday': [
        "New Year's Day", "Australia Day", "Good Friday", "Easter Saturday", "Easter Sunday", 
        "Easter Monday", "Anzac Day", "King's Birthday", "Labour Day", "Christmas Day", "Boxing Day"
    ],
    '2020': [
        '2020-01-01', '2020-01-27', '2020-04-10', '2020-04-11', '2020-04-12', 
        '2020-04-13', '2020-04-25', '2020-06-08', '2020-10-05', '2020-12-25', '2020-12-26'
    ],
    '2021': [
        '2021-01-01', '2021-01-26', '2021-04-02', '2021-04-03', '2021-04-04', 
        '2021-04-05', '2021-04-25', '2021-06-14', '2021-10-04', '2021-12-25', '2021-12-26'
    ],
    '2022': [
        '2022-01-01', '2022-01-26', '2022-04-15', '2022-04-16', '2022-04-17', 
        '2022-04-18', '2022-04-25', '2022-06-13', '2022-10-03', '2022-12-25', '2022-12-26'
    ],
    '2023': [
        '2023-01-01', '2023-01-26', '2023-04-07', '2023-04-08', '2023-04-09', 
        '2023-04-10', '2023-04-25', '2023-06-12', '2023-10-02', '2023-12-25', '2023-12-26'
    ],
    '2024': [
        '2024-01-01', '2024-01-26', '2024-03-29', '2024-03-30', '2024-03-31', 
        '2024-04-01', '2024-04-25', '2024-06-10', '2024-10-07', '2024-12-25', '2024-12-26'
    ],
    '2025': [
        '2025-01-01', '2025-01-27', '2025-04-18', '2025-04-19', '2025-04-20', 
        '2025-04-21', '2025-04-25', '2025-06-09', '2025-10-06', '2025-12-25', '2025-12-26'
    ]
}

# Step 6: Convert public holiday data to a DataFrame
df_holidays = pd.DataFrame(public_holidays)
df_holidays_melted = df_holidays.melt(id_vars=['holiday'], var_name='Year', value_name='Date')
df_holidays_melted = df_holidays_melted.dropna()
df_holidays_melted['Date'] = pd.to_datetime(df_holidays_melted['Date'])
df_holidays_melted['public_holiday'] = 1

# Step 7: Merge public holidays into the date range DataFrame
df_final = pd.merge(df_dates, df_holidays_melted[['Date', 'public_holiday']], left_on='datetime', right_on='Date', how='left')
df_final['public_holiday'] = df_final['public_holiday'].fillna(0).astype(int)

# Step 8: Drop unnecessary 'Date' column
df_holiday = df_final.drop(columns=['Date'])

# Step 9: Find the first date in df_holiday
first_date = df_holiday.index.min()

# Step 10: Filter df_final to include only dates after the first date in df_holiday
df_holiday_append = df_holiday.set_index('datetime')

# Step 11: Append the filtered holiday data to daily_traffic_data
daily_traffic_data.update(df_holiday_append[['school_holiday', 'public_holiday']])


